In [1]:
import requests
import re
import os
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

import selenium
from selenium import webdriver

## 접속 URL 및 설정

In [2]:
URL = 'https://www.incruit.com/company'

driver = webdriver.Chrome(executable_path='chromedriver')
driver.implicitly_wait(3)

driver.get(url=URL)

In [3]:
f = open('text1.txt')
corpList = f.read().split('\n')
f.close

<function TextIOWrapper.close()>

## 인재상, 핵심가치 검색후 가져오기

In [4]:
def get_data(corpName):
    enterCount = 0
    wantPeoples = []
    coreValues = []
    
    # 회사 이름으로 찾기
    start_page = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[2]/div[2]/div[1]/div[1]/p/input[1]')
    start_page.send_keys(corpName+'\n')
    time.sleep(0.2)
    enterCount += 1
    
    # 회사 페이지로 들어가기
    corpLinkText = driver.find_elements_by_class_name('listInfo')
    corpLinkTextList = [i.find_element_by_tag_name('a').get_attribute('textContent').strip() for i in corpLinkText]
    
    corp_idx = 0
    for i, cltl in enumerate(corpLinkTextList):
        if corpName in cltl:
            corp_idx = i
            break
    corpLink = corpLinkText[corp_idx].find_element_by_tag_name('a')
    
    # 회사 이름이 있으면 조건 추가
    a_corpName = corpLink.get_attribute('textContent').lower()
    
    if corpName not in a_corpName:
        print("기업이름이 다름니다. 찾으려는 기업: {}, 인크루트의 기업이름: {}".format(corpName, a_corpName))
        for i in range(enterCount):
            driver.back()
        return -1, corpName, a_corpName
    
    corpLink.send_keys('\n')
    time.sleep(0.2)
    enterCount += 1
    
    # 회사 페이지에서 메뉴텝에서 취업TIP이 있나 확인
    menuTap = driver.find_element_by_class_name('tabmenu_temp')
    menuList = menuTap.find_elements_by_tag_name('li')
    menuTextList = [menu.get_attribute('textContent').strip() for menu in menuTap.find_elements_by_tag_name('span')]

    if '취업TIP' in menuTextList:
        idx = menuTextList.index('취업TIP')
        menuList[idx].find_element_by_tag_name('a').send_keys('\n')
        time.sleep(0.2)
        enterCount += 1
    else:
        print(corpName, ": 취업TIP not exist")
        for i in range(enterCount):
            driver.back()
        return -2, corpName, a_corpName
    
    # 취업TIP에서 인재상 찾기 또는 핵심가치 찾기
    contents = driver.find_elements_by_class_name('contPre')
    for i, c in enumerate(contents):
        # 표로 되어 있는 경우
        try:
            rightpeopleClass = c.find_element_by_class_name('gongche_con_rightpeople')
            rightpeopletdTags = rightpeopleClass.find_elements_by_tag_name('td')
            for i in rightpeopletdTags:
                wantPeoples.append(i.get_attribute('textContent'))
        except:
            pass
        try:
            corvalueClass = c.find_element_by_class_name('gongche_con_corevalue')
            corvaluetdTags = corvalueClass.find_elements_by_tag_name('td')
            for i in corvaluetdTags:
                coreValues.append(i.get_attribute('textContent'))
        except:
            headName = c.find_elements_by_tag_name('h3')
            headName = [name.get_attribute('textContent').strip() for name in headName]
            if not(('인재상' in headName) or ('핵심가치' in headName)):
                continue

            headContent = c.find_elements_by_class_name('contPresent')
            headContent = [name.get_attribute('textContent').strip() for name in headContent]

            if '인재상' in headName:
                idx = headName.index('인재상')
                wantPeoples = headContent[idx].strip().split('/')
                wantPeoples = [wantPeople.strip() for wantPeople in wantPeoples]
            else:
                print(corpName, '인재상 not exist')

            if '핵심가치' in headName:
                idx = headName.index('핵심가치')
                coreValues = headContent[idx].strip().split('/')
                coreValues = [coreValue.strip() for coreValue in coreValues]
            else:
                print(corpName, '핵심가치 not exist')
    
    for i in range(enterCount):
        driver.back()
    
    return corpName, wantPeoples, coreValues

In [5]:
noCorp = []
diff_corpName = []
noCorpInfo = []
wantNValue = {}

for i, corp in enumerate(corpList):
    print(i+1,' ',end='')
    try:
        state, wantPeoples, coreValues = get_data(corp)
        # 정상 출력
        if state == corp:
            wantNValue[corp] = (wantPeoples, coreValues)
            print("success get data: ", corp)
        # 이름이 다른 경우
        elif state == -1:
            diff_corpName.append((wantPeoples, coreValues))
        # 취업TIP이 없는 경우
        elif state == -2:
            noCorpInfo.append(wantPeoples)
            
    except Exception as e:
        noCorp.append(corp)
        print("{}: {}".format(corp, e))
        driver.back()
        time.sleep(0.2)

1  11번가: Message: no such element: Unable to locate element: {"method":"css selector","selector":".tabmenu_temp"}
  (Session info: chrome=92.0.4515.159)

2  aia생명보험: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div[1]/div[2]/div[2]/div[1]/div[1]/p/input[1]"}
  (Session info: chrome=92.0.4515.159)

3  success get data:  aj네트웍스
4  akis: list index out of range
5  기업이름이 다름니다. 찾으려는 기업: bgf리테일, 인크루트의 기업이름: (주)gs리테일
6  기업이름이 다름니다. 찾으려는 기업: bnk투자증권, 인크루트의 기업이름: 대한무역투자진흥공사
7  기업이름이 다름니다. 찾으려는 기업: br코리아, 인크루트의 기업이름: 데상트코리아(주)
8  기업이름이 다름니다. 찾으려는 기업: cj4d플렉스, 인크루트의 기업이름: cj제일제당(주)
9  cjcgv: list index out of range
10  cjem: list index out of range
11  cjenm: list index out of range
12  기업이름이 다름니다. 찾으려는 기업: cj건설, 인크루트의 기업이름: (주)케이씨씨건설
13  기업이름이 다름니다. 찾으려는 기업: cj대한통운, 인크루트의 기업이름: 씨제이대한통운(주)
14  기업이름이 다름니다. 찾으려는 기업: cj씨푸드, 인크루트의 기업이름: cj제일제당(주)
15  기업이름이 다름니다. 찾으려는 기업: cj올리브네트웍스, 인크루트의 기업이름: cj제일제당(주)
16  cj올리브영: Message: no such element: Unabl

In [6]:
import pickle

f = open("noCorp_raw.pickle", "wb")
pickle.dump(noCorp, f)
f.close()

f = open("diff_corpName_raw.pickle", "wb")
pickle.dump(diff_corpName, f)
f.close()

f = open("noCorpInfo_raw.pickle", "wb")
pickle.dump(noCorpInfo, f)
f.close()

f = open("wantNValue_raw.pickle", "wb")
pickle.dump(wantNValue, f)
f.close()

* 기업이름 예외
    * 11번가 -> 가장위에는 십일번가, 하지만 아래의 부동산 11번가를 찾아 들어감
    
* 인재상, 핵심가치 예외
    * 강남제비스코, 일진머티리얼즈

* 이마트24 확인해보기